# 12.2 벡터 데이터베이스 작동 원리

- KNN (K-Nearest Neighbor)
- ANN (Approximate Nearest Neighbor)

## 12.2.1 KNN 검색과 그 한계
- 검색하려는 벡터와 가장 가까운 K개의 이웃 벡터를 찾는 검색 방식

### 벡터 검색의 성능 지표
- 색인 단계: 메모리 사용량, 색인 시간
- 검색 단계: 검색 시간, 재현율(실제로 가장 가까운 K개의 정답 데이터 중 몇 개가 검색 결과로 반환됐는지 비율)

In [ ]:
# 예제 12.1

# !wget ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz
# !tar -xf sift.tar.gz
# !mkdir -p data/sift1M
# !mv sift/* data/sift1M

# 서버 오류로 인해 파일 다운로드 불가 아래 링크의 데이터 활용
# https://huggingface.co/datasets/qbo-odp/sift1m/tree/main

In [ ]:
# 예제 12.2

import psutil

def get_memory_usage_mb(): # 현재 프로세스가 사용하는 메모리 사용량을 메가바이트 단위로 반환
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss / (1024 * 1024)

import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

ds = DatasetSIFT1M() # DatasetSIFT1M 클래스를 사용해 데이터셋을 불러온다.

xq = ds.get_queries() # 검색 데이터
xb = ds.get_database() # 저장된 벡터 데이터
gt = ds.get_groundtruth() # 실제 정답 데이터

In [ ]:
# 예제 12.3

k=1
d = xq.shape[1]
nq = 1000 # 검색 데이터 수
xq = xq[:nq] # 검색 데이터

for i in range(1, 10, 2):
    start_memory = get_memory_usage_mb()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(d) # 인덱스 생성
    index.add(xb[:(i+1)*100000]) # 임베딩 색인
    end_indexing = time.time()
    end_memory = get_memory_usage_mb()

    t0 = time.time()
    D, I = index.search(xq, k) # 검색
    t1 = time.time()
    print(f"데이터 {(i+1)*100000}개:")
    print(f"색인: {(end_indexing - start_indexing) * 1000 :.3f} ms ({end_memory - start_memory:.3f} MB) 검색: {(t1 - t0) * 1000 / nq :.3f} ms")

### 장점
- 재현율 100%
### 단점
- 데이터셋이 커질수록 성능 지표가 떨어짐

## 12.2.2 ANN 검색이란
- ANN (Approximate Nearest Neighbor) 근사 최근접 이웃
- 대용량 데이터셋에서 주어진 쿼리 항목과 가장 유사한 항목을 효율적으로 찾는데 사용되는 기술
- 약간의 정확도를 희생하는 대신 더 빠른 검색 속도를 제공
- IVF (Inverted File Index)
    - 데이터셋 벡터들을 클러스터로 그룹화하는 근사 최근접 이웃 검색 알고리즘
    - 전체 데이터셋 대신 몇개의 클러스터만 검색하는 전략
    - 인덱싱 시점: 데이터를 가져와서 중심점을 형성
    - 쿼리 시점: 가장 가까운 중심점을 찾은 다음, 해당 중심점 내에서 가장 가까운 데이터 포인트를 찾음
- HNSW (Hierachical Navigable Small World)
    - 그래프 기반 인덱싱 구조
    - 상위 계층에는 연결이 적고 하위 계층에는 연결이 밀집된 다층 그래프를 구축한다.
    - 최상위 계층에는 연결이 적기 때문에 데이터 사이의 거리가 먼데, 한 번에 먼 거리를 이동할 수 있기 때문에 그래프를 빠르게 탐색할 수 있게 된다.
    - 쿼리 시점: 최상위 계층에서 시작한 다음 검색을 구체화 하기 위해 아래 계층으로 진행
- HNSW의 탐색 속도가 빠르면서 검색 성능이 뛰어나 가장 많이 활용되고 있다.

## 12.2.3 탐색 가능한 작은 세계(NSW)
- 간선을 통해 서로 연결된 노드끼리만 탐색이 가능하기 때문에 노드와 노드 사이에 얼마나 많은 간선을 연결할지가 검색 성능과 검색 속도에 영향을 미친다.
- 

## 12.2.4 계층 구조
